# ROBERTA TWEETS


# TRAIN : SET - 1

# VAL : 

> # SET - 2

> # SET - 3

> # test

In [1]:
!pip install -q ktrain

In [2]:
import ktrain
from ktrain import text 

In [3]:
import tensorflow as tf
import numpy as np 
import pandas as pd 

# Load data

In [4]:
set1 = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/set1.csv')
set2 = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/set2.csv')
set3 = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/set3.csv')

test = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/tweets_test_features.csv')

In [5]:
set1['target'] = set1['target'].apply(lambda x : 'disaster' if x==1 else 'not_a_disaster')
set2['target'] = set2['target'].apply(lambda x : 'disaster' if x==1 else 'not_a_disaster')

In [6]:
x_train = set1.text.values.tolist()
y_train = set1.target.values.tolist()

x_test = set2.text.values.tolist()
y_test = set2.target.values.tolist()

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# MODEL BUILDING

In [8]:
MODEL_NAME = 'roberta-base'

In [9]:
t = text.Transformer(MODEL_NAME, maxlen=500)

In [10]:
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)


preprocessing train...
language: en
train sequence lengths:
	mean : 15
	95percentile : 24
	99percentile : 27


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 15
	95percentile : 24
	99percentile : 27


In [11]:
model = t.get_classifier()



# Train: set- 1
# Val : set- 2

In [12]:
learner = ktrain.get_learner(model, train_data=trn,val_data=val, batch_size=6) #predictor = ktrain.get_predictor(learner.model, preproc=t)

In [13]:
learner.fit_onecycle(lr = 2e-5, epochs = 2 )



begin training using onecycle policy with max lr of 2e-05...
Train for 635 steps, validate for 60 steps
Epoch 1/2
635/635 [==============================] - 577s 908ms/step - loss: 0.5261 - accuracy: 0.7270 - val_loss: 0.4250 - val_accuracy: 0.8177
Epoch 2/2
635/635 [==============================] - 561s 883ms/step - loss: 0.3675 - accuracy: 0.8518 - val_loss: 0.3908 - val_accuracy: 0.8397


# F1 SCORE

In [15]:
learner.validate(class_names = ['Disaster','not_a_disaster'])

                precision    recall  f1-score   support

      Disaster       0.86      0.75      0.80       817
not_a_disaster       0.83      0.91      0.87      1086

      accuracy                           0.84      1903
     macro avg       0.84      0.83      0.83      1903
  weighted avg       0.84      0.84      0.84      1903



array([[613, 204],
       [101, 985]])

# Predict Proba

In [16]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [18]:
x2 = set2.text.values
x3 = set3.text.values
test_data = test.text.values

In [19]:
x2_proba_arr = predictor.predict(x2, return_proba=True)

In [20]:
x3_proba_arr = predictor.predict(x3, return_proba=True)

In [21]:
test_proba_arr = predictor.predict(test_data, return_proba=True)

In [22]:
x2_proba_list = x2_proba_arr.tolist()
x3_proba_list = x3_proba_arr.tolist()
test_proba_list = test_proba_arr.tolist()

In [24]:
disaster_probability = []
not_a_disaster_probability = []
for i in x2_proba_list:
  disaster_probability.append(i[0])
  not_a_disaster_probability.append(i[1])

x2_df = pd.DataFrame({'Roberta_1': disaster_probability, 'Roberta_0': not_a_disaster_probability})


In [25]:
disaster_probability = []
not_a_disaster_probability = []
for i in x3_proba_list:
  disaster_probability.append(i[0])
  not_a_disaster_probability.append(i[1])

x3_df = pd.DataFrame({'Roberta_1': disaster_probability, 'Roberta_0': not_a_disaster_probability})

In [26]:
disaster_probability = []
not_a_disaster_probability = []
for i in test_proba_list:
  disaster_probability.append(i[0])
  not_a_disaster_probability.append(i[1])

test_df = pd.DataFrame({'Roberta_1': disaster_probability, 'Roberta_0': not_a_disaster_probability})

# Save Probas to drive

In [28]:
x2_df.to_csv('set2_Roberta_proba.csv', index=False) 
!cp set2_Roberta_proba.csv "/content/drive/My Drive/Tweets/5. Predict Probas" 

x3_df.to_csv('set3_Roberta_proba.csv', index=False) 
!cp set3_Roberta_proba.csv "/content/drive/My Drive/Tweets/5. Predict Probas" 

test_df.to_csv('test_data_Roberta_proba.csv', index=False) 
!cp test_data_Roberta_proba.csv "/content/drive/My Drive/Tweets/5. Predict Probas" 

# Sample of the files saved

In [30]:
x2_df[:2]

,Roberta_1,Roberta_0
0,0.188238,0.811762
1,0.992721,0.007279


In [31]:
x3_df[:2]

,Roberta_1,Roberta_0
0,0.073738,0.926262
1,0.403955,0.596045


In [32]:
test_df[:2]

,Roberta_1,Roberta_0
0,0.986153,0.013847
1,0.858267,0.141733
